In [1]:
import speech_recognition as sr
import pandas as pd

In [2]:
!gcloud auth application-default login
!gcloud auth application-default set-quota-project sunny-lightning-392907

Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=764086051850-6qr4p6gpi6hn506pt8ejuq83di341hur.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8085%2F&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Faccounts.reauth&state=5KJSnXAc8qntt42t92IvKp9KTOftfX&access_type=offline&code_challenge=R-YlGTNXUK8ZCNX7QHrgzYbWdbFycf-nyzNwjBjmIIs&code_challenge_method=S256


Credentials saved to file: [C:\Users\Vadim\AppData\Roaming\gcloud\application_default_credentials.json]

These credentials will be used by any library that requests Application Default Credentials (ADC).

Quota project "sunny-lightning-392907" was added to ADC which can be used by Google client libraries for billing and quota. Note that some services may still bill the proje

In [3]:
context = """
You are an assistant who helps users understand more about our application. You need to answer in one or two sentences.
Users will ask you questions and you will answer them given the below information. Don't answer in more than three sentences:

About the application:
Our application is an outlook plugin which helps users to analyze and write their emails more efficiently. it has the following functions:
1. It can show a sentiment of email, i.e. negative or positive. This will help users to avoid sending negative emails and try to write more positive emails
2. It can provide a language tone of email, this should help users to avoid impolite words and avoid using inappropriate language
3. It can summarize long emails in a few sentences. This can save time for users and avoid reading long email chains
4. It can rewrite an email in more polite and positive language.
5. It can translate to other languages

User question:
"""

In [4]:
from vertexai.preview.language_models import TextGenerationModel
import vertexai

def send_user_input(project_id, location, question, temperature: float = .1):
    vertexai.init(project=project_id, location=location)
    
    parameters = {
        "temperature": temperature,
        "max_output_tokens": 64,
        "top_p": .8,
        "top_k": 40,
    }

    model = TextGenerationModel.from_pretrained("text-bison@001")
    request_text = context + question 
    #print(request_text)
    response = model.predict(request_text, **parameters)
    #print(f"Response from Model: {response.text}")
    return response.text
   


cloudpickle is not installed. Please call `pip install google-cloud-aiplatform[preview]`.


In [7]:
df = pd.read_excel("Answers/HackathonQuestionsExcel.xlsx",header=0,converters={'id':str})


In [8]:
questions = list(df.iloc[:, 1].values)
questions

["What's the name of the Banking Unicorn?",
 'How does the Banking Unicorn work?',
 'Can the Banking Unicorn learn and improve its responses over time?',
 'Is the Banking Unicorn based on a specific AI technology or framework?',
 "What's the purpose of having an Banking Unicorn at your exhibition stand?",
 'Can the Banking Unicorn interact with other AI technologies or devices?',
 'Is the Banking Unicorn customizable in any way?',
 "What are the limitations of the Banking Unicorn's capabilities?",
 'Can I take a picture with the Banking Unicorn?',
 'What is MailboxIQ and what does it do?',
 'What inspired you to develop the MailboxIQ App?',
 'How can the MailboxIQ App help individuals or businesses?',
 'Is the MailboxIQ App available for download or use?',
 'What platforms or devices is the MailboxIQ App compatible with?',
 'Can you provide some key features or benefits of the MailboxIQ App?',
 'Are there any success stories or use cases for the MailboxIQ App?',
 'How do you ensure the

In [9]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

model = SentenceTransformer('all-MiniLM-L6-v2')
#model = SentenceTransformer('stsb-roberta-large')

embeddings = model.encode(questions)

In [10]:
def get_score(model, test_sentence, questions):
    test_emb = model.encode(test_sentence)
    test_emb = test_emb.reshape(1, -1)
    max_cs = 0
    max_idx = -1
    for i, emb in enumerate(embeddings):
        cs = cosine_similarity(test_emb, emb.reshape(1, -1))[0][0]
        if cs > max_cs:
            max_cs = cs
            max_idx = i    
    return (max_cs, max_idx)
print(get_score(model, "What's MailboxIQ?", questions))

(0.94933844, 9)


In [11]:
output_voice_file="response_new2.mp3"
output_voice_file_changed = "response_new2_changed.mp3"
idle_video_file = "idle9s.mp4"
#input_video_file = "listen.mp4"
listen_video_file = "listen_long.mp4"
speaking_video_file = "video_assistant.mp4"

In [26]:
#Vadim's Laptop
screen_size = [1920,1080]
micro_pos = [1300,710]
space_key_text_pos = [450,800]
speak_text_pos = [800,800]
ask_me_text_pos = [1370, 200]

sample_questions_img_pos = [1360, 180]
video_scale = 1.0

# predefined questions, similarity threshold
similarity_threshold = 0.5

# parameters
voice_shift = 7
adjust_for_ambient_noise_duration = 0.2
log_file = 'conversation.log'

In [27]:
import pygame
from pygame.locals import *
import cv2
import numpy as np
import sys
from gtts import gTTS
from playsound import playsound
import cv2
import librosa
import soundfile as sf
import IPython
from IPython.display import Audio
from threading import Thread
from enum import Enum
from pygame import mixer
import os
import speech_recognition as sr
import shutil

def log(msg):
    print(msg)
    with open(log_file, 'a') as f:
        f.write(msg+'\n')
    
class VoiceRecordingThread(Thread):
    def run(self):
        self.response = ""
        r = sr.Recognizer()
        with sr.Microphone(device_index=0) as source:
            r.adjust_for_ambient_noise(source, duration=adjust_for_ambient_noise_duration)
            #frame = printText(frame, "speak")
            #cv2.imshow('Unicorn', frame)
            #quitButton = cv2.waitKey(25) & 0xFF == ord('q')
            log("speak")
            audio = r.listen(source)
        user_input = r.recognize_google(audio)
        log("User input:", user_input)
        score, idx = get_score(model, user_input, questions)
        log("scores:", score, idx)
        if score > similarity_threshold:
            log("use predefined question: ", questions[idx])
            file_id = df.iloc[idx, 0]
            shutil.copyfile("Answers/"+file_id+"_response_unicorn.mp3", output_voice_file_changed)
            self.response = df.iloc[idx, 2]
        else:
            self.response = send_user_input('sunny-lightning-392907', 'us-central1', user_input)
            log("Model response:", self.response)
            voice_prep(self.response)
     
    
class PlaySoundThread(Thread):
    def run(self):
        mixer.init()
        mixer.music.load(output_voice_file_changed)
        mixer.music.play()
        while(mixer.music.get_busy()):
            pass
        mixer.music.unload()
        log("audio end")

        
        
def voice_prep(response):
    language = 'en'

    speech = gTTS(text=response, lang=language)

    speech.save(output_voice_file)
    save_shifted_pitch(output_voice_file, output_voice_file_changed, voice_shift)
    #save_shifted_pitch("response", 7, )
    #os.system("start response_shifted.wav")
    #os.system("start " + output_voice_file)
    #playsound(output_voice_file)

def printText(frame, text, pos = 50):
    font = cv2.FONT_HERSHEY_SIMPLEX
    org = (pos, 1000)
    fontScale = 2
    color = (0, 255, 255)
    thickness = 4
    frame = cv2.putText(frame, text, org, font, 
                       fontScale, color, thickness, cv2.LINE_AA)
    return frame
    
class States(Enum):
    IDLE = 1
    LISTENING = 2
    SPEAKING = 3

def unicornVideo():
    pygame.init()

    # prepare assets

    micro_img_white = pygame.image.load('micro_white.png')
    micro_img_black = pygame.image.load('micro.png')
    micro_img = micro_img_black

    big_font = pygame.font.SysFont('comicsansms', 48)
    space_key_text = big_font.render('Press and hold Space key to speak', True, (0,0,0))
    speak_text = big_font.render('please speak', True, (0,0,0))

    small_font = pygame.font.SysFont('comicsansms', 24)
    ask_me_text = small_font.render('What can you ask me?', True, (255,255,255))
    sample_questions_img = pygame.image.load('sample_questions.png')

    # current video

    cap = cv2.VideoCapture(idle_video_file)

    #pygame.display.set_caption("OpenCV camera stream on Pygame")
    screen = pygame.display.set_mode(screen_size)
    shape = None
    
    #
    
    speaking_thread = None
    listening_thread = None
    state = States.IDLE
    text_response = None   
    
    try:
        while(cap.isOpened()):
            if state == States.LISTENING:
                if not listening_thread.is_alive():
                    #print("thread not alive")
                    text_response = listening_thread.response
                    listening_thread = None
                    state = States.SPEAKING
                    speaking_thread = PlaySoundThread()
                    if text_response:
                        speaking_thread.start()
                        cap.release()
                        cap = cv2.VideoCapture(speaking_video_file)
            elif state == States.SPEAKING:
                if not speaking_thread.is_alive():
                    speaking_thread = None
                    state = States.IDLE
                    cap.release()
                    cap = cv2.VideoCapture(idle_video_file)
                    
            ret, frame = cap.read()
            if ret:
                screen.fill([0,0,0])
                shape = (frame.shape[0]*video_scale, frame.shape[1]*video_scale)
                frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                frame = np.rot90(frame)
                frame = pygame.surfarray.make_surface(frame)
                if video_scale != 1:
                    frame = pygame.transform.scale_by(frame, video_scale)

                if screen_size[0] > shape[0]:
                    screen.blit(frame, ( (screen_size[0] - shape[0])/2,0))
                else:
                    screen.blit(frame, (0,0))

                #display UI
                screen.blit(micro_img, micro_pos)

                if state == States.IDLE:
                    screen.blit(space_key_text, space_key_text_pos)
                elif state == States.LISTENING:
                    screen.blit(speak_text, speak_text_pos)
                    
                    

                #screen.blit(ask_me_text, ask_me_text_pos)

                screen.blit(sample_questions_img, sample_questions_img_pos)
                # update

                pygame.display.update()

                for event in pygame.event.get():
                    if event.type == KEYDOWN:
                        if event.key == pygame.K_SPACE and state == States.IDLE:
                            micro_img = micro_img_white
                            state = States.LISTENING
                            log("space")
                            cap.release()
                            cap = cv2.VideoCapture(listen_video_file)

                            listening_thread = VoiceRecordingThread()
                            listening_thread.start()
                            log("listening thread is started")

                        elif event.key == pygame.K_q:
                            sys.exit(0)
                    elif event.type == KEYUP:
                        if event.key == pygame.K_SPACE:
                            micro_img = micro_img_black
                    elif event.type == pygame.QUIT:
                        sys.exit(0)

                cv2.waitKey(25)
            else:
                if state == States.IDLE:
                    cap = cv2.VideoCapture(idle_video_file)
                elif state == States.LISTENING:
                    cap = cv2.VideoCapture(listen_video_file)
                else:
                    cap = cv2.VideoCapture(speaking_video_file)
                    
    except (KeyboardInterrupt, SystemExit):
        pygame.quit()
        cv2.destroyAllWindows()

        

def shift_pitch(filename, higher_pitch_shift, lower_pitch_shift, load_original=False):
    audio_wavelength, sampling_rate = librosa.load(filename)
    if load_original:
        IPython.display.display(Audio(data=audio_wavelength, rate=sampling_rate, autoplay=False))
    if higher_pitch_shift > 0:
        higher_pitch = librosa.effects.pitch_shift(audio_wavelength, sr=sampling_rate, n_steps=higher_pitch_shift)
        IPython.display.display(Audio(data=higher_pitch, rate=sampling_rate, autoplay=False))
    if lower_pitch_shift < 0:
        lower_pitch = librosa.effects.pitch_shift(audio_wavelength, sr=sampling_rate, n_steps=lower_pitch_shift)
        IPython.display.display(Audio(data=lower_pitch, rate=sampling_rate, autoplay=False))
        
def save_shifted_pitch(output_voice_file, output_voice_file_changed, pitch_shift, load_original=False, load_altered=False):
    audio_wavelength, sampling_rate = librosa.load(output_voice_file)
    altered = librosa.effects.pitch_shift(audio_wavelength, sr=sampling_rate, n_steps=pitch_shift)
    sf.write(output_voice_file_changed, altered, sampling_rate)

In [28]:
if __name__ == '__main__':
    unicornVideo()

Exception in thread Thread-13:
Traceback (most recent call last):
  File "C:\Users\Vadim\AppData\Local\Temp\ipykernel_15380\1272385917.py", line 30, in run
  File "D:\workspace\dev\conda\envs\assistant\Lib\site-packages\speech_recognition\__init__.py", line 383, in adjust_for_ambient_noise
    assert source.stream is not None, "Audio source must be entered before adjusting, see documentation for ``AudioSource``; are you using ``source`` outside of a ``with`` statement?"
           ^^^^^^^^^^^^^^^^^^^^^^^^^
AssertionError: Audio source must be entered before adjusting, see documentation for ``AudioSource``; are you using ``source`` outside of a ``with`` statement?

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "D:\workspace\dev\conda\envs\assistant\Lib\threading.py", line 1038, in _bootstrap_inner
    self.run()
  File "C:\Users\Vadim\AppData\Local\Temp\ipykernel_15380\1272385917.py", line 29, in run
  File "D:\workspace\d

space
listening thread is started


In [21]:
voice_prep("hello")

In [22]:
PlaySoundThread().start()

audio end


In [ ]:
questions

In [ ]:
fonts = pygame.font.get_fonts()
print(len(fonts))
for f in fonts:
    print(f)

## TODO:
* log